In [5]:
import os, random, gzip, json, re
import numpy as np
import sentence_transformers
from typing import List, Tuple, Dict

DIALOGUE_FILE = "en-comedy.txt.gz"
FIRST_NAMES = "first_names.json"
SBERT_MODEL = "all-MiniLM-L6-v2"

model = sentence_transformers.SentenceTransformer(SBERT_MODEL)

c:\Users\sanfe\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
pairs = []
input_text = None
dialogue_started = False

with open("first_names.json", "r") as names_file:
    name_list = [name.strip() for name in names_file.readlines()][0]
    names_list = name_list.replace(", ", "").replace('""', '"').split('"')[1:-1]

with gzip.open(DIALOGUE_FILE, "rt", encoding = "utf-8") as file:
    for line in file:
        line = line.strip()

        if line.startswith("###"):
            dialogue_started = True
            input_text = None
            continue
        
        if not dialogue_started:
            continue
            
        response_text = line

        if input_text and not bool(re.search(r'[,\(\)\[\]\{\}":;]', input_text)) and not input_text.isupper() \
                and not len(input_text.split()) > 10 and not any(word in name_list for word in input_text.split()) \
                and not len(input_text.split()) == 1:
            pairs.append((input_text, response_text)) 

        input_text = response_text

        if len(pairs) > 20:
            print(pairs)
            break
    

[("That's first.", 'And then I will feel like I need to pee a little bit, and often times I do, just a little.'), ("I'm just claustrophobic.", "I'll feel like..."), ("I'll feel like...", "Like, I'm trapped, like I can't go nowhere, you know, and I'll start getting like, all desperate, like, I need to get out of here."), ('Bird flu.', "Nobody's going to get bird flu."), ('Why clean?', "It only gets dirty again, that's my theory."), ("I'm Mrs. Messy.", 'Why do I think some people are scared of bugs?'), ('Because they...', "They don't know any better."), ("That's psychobabble.", "That's doubletalk."), ("That's doubletalk.", '"The only thing you have to be afraid of is being afraid."'), ('Right there.', "There's the immaturity."), ('Talk intelligent.', '- You sound like an idiot.'), ("Don't make them...", 'That "special" makes them sound like they got something wrong with them.'), ('That was...', 'I was just trying to be funny.'), ("What's wrong with kids today?", 'A lot of things are wron

In [17]:
embeddings = []

for input_text, response_text in pairs:
    # compute the embedding using self.model and then add it to the list
    embedding_input = model.encode(input_text)
    embeddings.append(embedding_input)
    
# transform the list of embeddings into a matrix
embeddings_matrix = np.vstack(embeddings)

print(embeddings_matrix)

[[-0.02892047 -0.09175336 -0.03663573 ...  0.06204863  0.14450513
  -0.02157983]
 [ 0.12187161 -0.10628247  0.03366105 ...  0.01816681 -0.02022657
  -0.05941494]
 [-0.11259668 -0.0756629   0.01430398 ... -0.02680683 -0.03479368
  -0.07686378]
 ...
 [ 0.08680858  0.00744335  0.03650837 ... -0.00873115  0.0020918
  -0.03564541]
 [-0.05748695 -0.05801114  0.05182722 ...  0.10805812  0.10157779
  -0.04138391]
 [-0.01507761 -0.06479713  0.06085821 ...  0.08318344  0.01034151
  -0.09962135]]


In [22]:
from scipy import spatial

user_utterance = ["I think I am lost."]
user_embedding = np.squeeze(model.encode(user_utterance))

# Compute cosine similarity between the user's embedding and all input embeddings
similarities = 1 - np.apply_along_axis(lambda x: spatial.distance.cosine(user_embedding, x), 1, embeddings_matrix)

# Find the index of the most similar input
most_similar_index = np.argmax(similarities)
    
# Retrieve and return the corresponding response
response = pairs[most_similar_index][1]
print(response)

I got them thick nails so when you cut them, you might wanna put a pair of goggles on, and stand back a minute, 'cause them babies might start popping and it's on.
